In [45]:
%load_ext autoreload
%autoreload 2

from webbrowser import get
import pandas as pd
from sklearn.metrics import accuracy_score, r2_score
from models import *

from pretty_logger import get_logger
from pathlib import Path
from utils import get_git_root

project_root = get_git_root()

logger = get_logger(name="modelslog", full_path=Path(project_root, 'log/models.log'), add_console_hander=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:

df = pd.read_csv(datafile)
df["date"] = pd.to_datetime(df["day_survey"])
df['C'] = df['date'].apply(lambda date: COVIDStatus.POST_COVID if date > date_covid else COVIDStatus.PRE_COVID)

df_head = df.head(5).copy()

df.rename(columns=reverse_ema_dictionary, inplace=True)
df.set_index(["uid", "date"], inplace=True)
df.dropna(subset=ema + physical + social + sleep, inplace=True)

sets_df = pd.read_parquet(sets_file, engine='pyarrow')

In [47]:
import numpy as np
index = 51
model_rows = [sets_df.iloc[index]]
#for index, model_row in sets_df.iterrows():

n_estimatorss = [20]
ccp_alphas = np.logspace(-3, 0, 4)
# list of sets
fitted_sets_of_covariates = []


for ccp_alpha in ccp_alphas:
    for n_estimators in n_estimatorss:
        #for model_row in model_rows:
        for index, model_row in sets_df.iterrows():
            logger.info("\n")
            logger.info(f"Modeling treatment:{full_dictionary[model_row['treatment']]} on outcome:{full_dictionary[model_row['outcome']]}")
            logger.info(f"Covariates: {set(all_covariates)}")

            # people can't change their demographics.. except maybe OS :)
            if model_row['treatment'].startswith('D'):
                logger.info(f"Skippnig demographic treatment {model_row['treatment']}")
                continue

            if model_row['outcome'] not in ["Y2", "Y3"]:
                logger.info("Only doing PHQ right now")
                continue
            all_covariates = [model_row['treatment']] + model_row['sets'].tolist()
            if set(all_covariates) in fitted_sets_of_covariates:
                logger.info(f"already have done {set(all_covariates)}")
                continue
            wbm = WBRandomForestModel(
                data=df,
                n_estimators=n_estimators,
                ccp_alpha=1e-4,
                #gamma=gamma,
                #alpha=alpha,
                #activation='relu',
                #max_iter=1000,
                #hidden_layer_sizes=(32,8,4),
                treatment=model_row['treatment'],
                outcome=model_row['outcome'],
                separating_set=model_row['sets'].tolist(),
                name=f"row:{index}"
                )
            fitted_sets_of_covariates.append(set(wbm.model_covariates))
            logger.info(f"ccp_alpha ={ccp_alpha}, n_estimators={n_estimators}, pre_rsq={wbm.pre_r_squared} post_r_sq={wbm.post_r_squared}")
            logger.info("-----------------------------------------------------")


Binarizing using median value (excercise median=1.10e+04)
Directory 20240528_003338_135888 created.
Binarizing using median value (studying median=2.72e+00)
Directory 20240528_003338_997024 created.
Binarizing using median value (in house median=1.14e+01)
Directory 20240528_003339_744156 created.
Binarizing using median value (sports median=8.33e-02)
Directory 20240528_003340_396176 created.
Binarizing using median value (traveling median=8.54e+02)
Directory 20240528_003341_144906 created.
Binarizing using median value (time in social location median=1.27e+00)
Directory 20240528_003341_929109 created.
Binarizing using median value (duration unlocked phone in social locations median=1.03e+01)
Directory 20240528_003342_851687 created.
Binarizing using median value (frequency of unlocked phone in social locations median=8.96e+00)
Directory 20240528_003343_769631 created.
Binarizing using median value (in house median=1.14e+01)
Directory 20240528_003344_659468 created.
Binarizing using med

In [48]:
results_df = aggregate_results_from_subfolders('.')
results_df.to_csv('results.csv')